In [42]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from xgboost import XGBClassifier
import pandas as pd

data = pd.read_csv('../mentalhealth.csv', encoding='cp1252')
cv = CountVectorizer(max_features=5000, ngram_range = (1,3), token_pattern = "[A-Za-z_][A-Za-z\d_]*")
X = cv.fit_transform(data.Text).toarray()
y = data['MH']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)
count_df = pd.DataFrame(X_train, columns=cv.get_feature_names())

data.sample(5)

,Text,MH
111,i am a student in need of mental help \n,1
196,Financial aid for cancer,0
120,Filipino foreigner on short term visit pass ne...,0
264,Covid-19 recovery schemes,0
15,Pregnant teen need counselling,1


In [43]:
from bs4 import BeautifulSoup
import re
def cleanText(text):
    text = BeautifulSoup(text, "lxml").text
    text = re.sub(r'\|\|\|', r' ', text) 
    text = re.sub(r'http\S+', r'<URL>', text)
    text = text.lower()
    text = text.replace('x', '')
    return text

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
porter = PorterStemmer()
def stemSentence(sentence):
    token_words=word_tokenize(sentence)
    token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

data['Text'] = data['Text'].apply(cleanText).apply(stemSentence)

In [44]:
data

,Text,MH
0,my client need marit counsel,1
1,counsel,1
2,"career counsel , youth",1
3,counsel,1
4,counsel in redhil,1
...,...,...
311,my client is retrench and ha no incom . she ne...,0
312,i have diabeti what can i do ?,0
313,dad unemploy,0
314,senior fell down and need interim caregiv,0


In [45]:
# fit model no training data

model = XGBClassifier()
model.fit(X_train, y_train)

# make predictions for test data

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

# evaluate predictions

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

c:\python38\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:46:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 80.95%


In [65]:
input_text = "i am emotionally drained and burn out"
cv_test = CountVectorizer(vocabulary=cv.get_feature_names())
inputtexttovectorize = pd.DataFrame([[input_text, 0]], columns = ['Text', 'MH'])

X_test = cv_test.fit_transform(inputtexttovectorize.Text)

In [64]:
model.predict_proba(X_test)

array([[0.60537386, 0.39462617]], dtype=float32)

In [67]:
inputtexttovectorize

,Text,MH
0,i am emotionally drained and burn out,0
